In [48]:
from IPython.display import Markdown, display
import re


def format_category_list(categories):
    """
    Format list of categories with proper grammar: cat1, cat2, or cat3.
    
    Args:
        categories: list - List of category names
        
    Returns:
        str - Formatted category list with backticks and proper grammar
    """
    if not categories:
        return ""
    
    # Format category names (lowercase, replace underscores with spaces)
    formatted_cats = [cat.lower().replace('_', ' ') for cat in categories]
    
    if len(formatted_cats) == 1:
        return f"`{formatted_cats[0]}`"
    elif len(formatted_cats) == 2:
        return f"`{formatted_cats[0]}` or `{formatted_cats[1]}`"
    else:
        # Multiple categories: cat1, cat2, or cat3
        cats_with_backticks = [f"`{cat}`" for cat in formatted_cats[:-1]]
        return ", ".join(cats_with_backticks) + f", or `{formatted_cats[-1]}`"


def bold_matching_portion(value, search_term):
    """
    Bold the portion of value that matches the search term.
    Handles special characters and edge cases safely.
    
    Args:
        value: str - The value to search within
        search_term: str - The term to find and bold
        
    Returns:
        str - Value with matching portion bolded, or original value if no match
    """
    if not value or not search_term:
        return value
    
    value_upper = value.upper()
    search_upper = search_term.upper()

    try:
        if search_upper in value_upper:
            start_idx = value_upper.index(search_upper)
            end_idx = start_idx + len(search_upper)
            
            before = value[:start_idx]
            match = value[start_idx:end_idx]
            after = value[end_idx:]
            
            return f"{before}**{match}**{after}"
    except (ValueError, IndexError) as e:
        # If any error occurs during matching, return original value
        return value
    
    return value


def get_top_matches(suggested_entities, limit=5, min_threshold=0):
    """
    Sort entities by similarity score and return top matches.
    
    Args:
        suggested_entities: list - Entities to sort and filter
        limit: int - Maximum number of matches to return (default 5)
        min_threshold: int - Minimum similarity score to include (default 0)
        
    Returns:
        list - Top matching entities above threshold
    """
    if not suggested_entities:
        return []
    
    # Filter by threshold first
    filtered = [e for e in suggested_entities if e.get("similarity_score", 0) >= min_threshold]
    
    # Sort and limit
    sorted_entities = sorted(
        filtered, 
        key=lambda x: (-x.get("similarity_score", 0), x.get("suggested_value", ""))
    )
    
    return sorted_entities[:limit]


def get_column_info(entities):
    """
    Get column name if all entities are from same column.
    
    Args:
        entities: list - Entities to analyze
        
    Returns:
        tuple - (all_same_column: bool, column_name: str or None)
    """
    if not entities:
        return False, None
    
    all_same_column = len(set(entity.get("column", "") for entity in entities)) == 1
    column_name = entities[0].get("column", "").lower() if all_same_column and entities else None
    return all_same_column, column_name


def format_bullet_list(entities, user_input, all_same_column):
    """
    Format entities as a bullet list with optional column labels.
    
    Args:
        entities: list - Entities to format
        user_input: str - Original user input for bolding
        all_same_column: bool - Whether all entities are from same column
        
    Returns:
        str - Formatted bullet list
    """
    if not entities:
        return ""
    
    lines = []
    for entity in entities:
        value = entity.get("suggested_value", "")
        if not value:
            continue
            
        value_formatted = value.title()
        value_with_bold = bold_matching_portion(value_formatted, user_input)
        
        if not all_same_column:
            column = entity.get("column", "")
            if column:
                column_formatted = format_category_list([column])
                lines.append(f"* {value_with_bold} {column_formatted}")
            else:
                lines.append(f"* {value_with_bold}")
        else:
            lines.append(f"* {value_with_bold}")
    
    return "\n".join(lines)


def format_entity_clarification(user_input, suggested_entities):
    """
    Format entity clarification message with bullet point options.

    Args:
        user_input: str - The user's original input (e.g., "CORONA")
        suggested_entities: list of dict - Suggested entities from similarity matching

    Returns:
        str - Formatted clarification message or fallback message if no matches
    """
    # Validation: Check if we have any entities
    if not suggested_entities:
        return f'I couldn\'t find any matches for **"{user_input}"** in the data. Please try rephrasing your question or check the spelling.'
    
    top_matches = get_top_matches(suggested_entities)
    
    # Validation: Check if we have matches after filtering
    if not top_matches:
        return f'I couldn\'t find any close matches for **"{user_input}"** in the data. Please try rephrasing your question or check the spelling.'
    
    all_same_column, column_name = get_column_info(top_matches)

    # Build header
    if all_same_column and column_name:
        # Format column name without backticks for use in header
        column_formatted = column_name.replace('_', ' ')
        header = f'I couldn\'t find **"{user_input}" {column_formatted}** in the data. Which one did you mean?\n\n'
    else:
        header = f'I couldn\'t find **"{user_input}"** in the data. Which one did you mean?\n\n'

    # Build bullet list
    bullet_list = format_bullet_list(top_matches, user_input, all_same_column)
    
    # Final validation: Check if we generated any bullets
    if not bullet_list:
        return f'I couldn\'t find any valid matches for **"{user_input}"** in the data. Please try rephrasing your question.'
    
    return header + bullet_list


def format_multiple_entities_clarification(entities_dict):
    """
    Format clarification for multiple entities at once.
    
    Args:
        entities_dict: dict - Dictionary with user inputs as keys and suggested entities as values

    Returns:
        str - Formatted clarification message for multiple entities
    """
    # Validation: Check if we have any entities
    if not entities_dict:
        return "I am sorry but I am not able to respond to your query right now as our systems are facing some issue. Our team has been notified to address this issue. Please Contact Admin for further support."
    
    # Filter out empty entity lists
    filtered_dict = {k: v for k, v in entities_dict.items() if v}
    
    if not filtered_dict:
        # return "I couldn't find any matches for the terms in your question. Please try rephrasing or check the spelling."
    
        # Collect terms that have no suggestions
        empty_terms = [term for term, entities in entities_dict.items() if not entities]
        
        # Mention empty terms in the message if any exist
        if empty_terms:
            if len(empty_terms) == 1:
                terms_str = f'**"{empty_terms[0]}"**'
                return f"I couldn't find any matches for the term {terms_str} in the data. Please try rephrasing your question or check the spelling."
            elif len(empty_terms) == 2:
                terms_str = f'**"{empty_terms[0]}"** and **"{empty_terms[1]}"**'
                return f"I couldn't find any matches for the terms {terms_str} in the data. Please try rephrasing your question or check the spelling."
            else:
                terms_list = [f'**"{term}"**' for term in empty_terms[:-1]]
                terms_str = ", ".join(terms_list) + f', and **"{empty_terms[-1]}"**'
                return f"I couldn't find any matches for the terms {terms_str} in the data. Please try rephrasing your question or check the spelling."
    
    if len(filtered_dict) == 1:
        # Single entity case
        user_input, entities = next(iter(filtered_dict.items()))
        return format_entity_clarification(user_input, entities)
    
    # Multiple entities case
    message = "I couldn't find exact matches for some terms in your question. Please clarify them so I can proceed.\n\n"
    
    sections = []
    for user_input, entities in filtered_dict.items():
        top_matches = get_top_matches(entities)
        
        # Skip if no matches after filtering
        if not top_matches:
            continue
            
        all_same_column, column_name = get_column_info(top_matches)
        
        # Build section header
        if column_name:
            # Format column name without backticks for use in header
            column_formatted = column_name.replace('_', ' ')
            section_header = f'**"{user_input}" {column_formatted}**:'
        else:
            section_header = f'**"{user_input}"**:'
        
        # Build bullet list
        bullet_list = format_bullet_list(top_matches, user_input, all_same_column)
        
        if bullet_list:  # Only add section if we have bullets
            sections.append(f"{section_header}\n{bullet_list}")
    
    # Validation: Check if we generated any sections
    if not sections:
        return "I couldn't find any valid matches for the terms in your question. Please try rephrasing or check the spelling."
    
    return message + "\n\n".join(sections)


# Example Usage
if __name__ == "__main__":
    # Example data from the new structure
    suggested_entities_case_1 = {
        "CORONA": [
            {
                "suggested_value": "CORONA CERO",
                "column": "BRAND",
                "similarity_score": 100,
            },
            {
                "suggested_value": "CORONA EXTRA",
                "column": "BRAND",
                "similarity_score": 100,
            },
            {
                "suggested_value": "CORONA GOLDEN LIGHT",
                "column": "SUB_BRAND",
                "similarity_score": 100,
            },
            {
                "suggested_value": "CORONA LIGERA",
                "column": "BRAND",
                "similarity_score": 100,
            },
            {
                "suggested_value": "CORONA LIGHT",
                "column": "BRAND",
                "similarity_score": 100,
            },
            {"similarity_score": 80, "column": "COUNTRY", "suggested_value": "USA"},
            {"similarity_score": 75, "column": "BRAND", "suggested_value": "BOON"},
            {"similarity_score": 75, "column": "BRAND", "suggested_value": "KONA"},
            {"similarity_score": 75, "column": "BRAND", "suggested_value": "TONA"},
            {
                "similarity_score": 73,
                "column": "BRAND",
                "suggested_value": "BLACK CROWN",
            },
            {"similarity_score": 73, "suggested_value": "GORDON", "column": "BRAND"},
        ]
    }
    suggested_entities_case_2 = {
        "BRAND BB": [
            {"suggested_value": "BRAND", "column": "BRAND", "similarity_score": 100},
            {"suggested_value": "ANDES", "column": "SUBBRAND", "similarity_score": 75},
            {
                "suggested_value": "ANOTHER BRAND",
                "column": "BRAND",
                "similarity_score": 75,
            },
        ],
        "REGION N": [
            {
                "similarity_score": 88,
                "column": "REGION",
                "suggested_value": "CENTRAL REGION",
            },
            {
                "similarity_score": 88,
                "column": "REGION",
                "suggested_value": "NORTH EAST REGION",
            },
            {
                "similarity_score": 100,
                "column": "REGION",
                "suggested_value": "REGION NORTH",
            },
            {
                "suggested_value": "NORTH WEST REGION",
                "similarity_score": 88,
                "column": "REGION",
            },
            {
                "suggested_value": "REGION PARISIENNE",
                "similarity_score": 88,
                "column": "REGION",    
            },    
            {    
                "suggested_value": "SOUTH REGION",    
                "similarity_score": 75,    
                "column": "REGION",    
            },    
        ],    
    }   


    # Format clarification messages    print("="*60 + "\n")
    print("CASE 1: Single Entity Clarification")
    print("\n" + "="*60)    
    clarification_msg_1 = format_multiple_entities_clarification(suggested_entities_case_1)
    print(clarification_msg_1)
    display(Markdown(clarification_msg_1))
    print("\n" + "="*60)
    print("CASE 2: Multiple Entities Clarification")
    print("="*60 + "\n")
    clarification_msg_2 = format_multiple_entities_clarification(suggested_entities_case_2)
    print(clarification_msg_2)
    display(Markdown(clarification_msg_2))

CASE 1: Single Entity Clarification

I couldn't find **"CORONA"** in the data. Which one did you mean?

* **Corona** Cero `brand`
* **Corona** Extra `brand`
* **Corona** Golden Light `sub brand`
* **Corona** Ligera `brand`
* **Corona** Light `brand`


I couldn't find **"CORONA"** in the data. Which one did you mean?

* **Corona** Cero `brand`
* **Corona** Extra `brand`
* **Corona** Golden Light `sub brand`
* **Corona** Ligera `brand`
* **Corona** Light `brand`


CASE 2: Multiple Entities Clarification

I couldn't find exact matches for some terms in your question. Please clarify them so I can proceed.

**"BRAND BB"**:
* Brand `brand`
* Andes `subbrand`
* Another Brand `brand`

**"REGION N" region**:
* **Region N**orth
* Central Region
* North East Region
* North West Region
* Region Parisienne


I couldn't find exact matches for some terms in your question. Please clarify them so I can proceed.

**"BRAND BB"**:
* Brand `brand`
* Andes `subbrand`
* Another Brand `brand`

**"REGION N" region**:
* **Region N**orth
* Central Region
* North East Region
* North West Region
* Region Parisienne

In [49]:
# Test Edge Cases and Validation

print("="*80)
print("EDGE CASE & VALIDATION TESTS")
print("="*80 + "\n")

# Test 1: Special characters in matching
print("Test 1: Special Characters (apostrophe)")
test_special = bold_matching_portion("O'DOUL'S AMBER", "O'DOUL")
print(f"Input: O'DOUL'S AMBER, Search: O'DOUL")
print(f"Result: {test_special}")
print(f"✅ PASS: Contains **O'DOUL**" if "**O'DOUL**" in test_special else "❌ FAIL")
print()

# Test 2: Empty entities list
print("Test 2: Empty Entities List")
result_empty = format_entity_clarification("CORONA", [])
print(f"Input: Empty list")
print(f"Result: {result_empty}")
print(f"✅ PASS: Fallback message shown" if "couldn't find any" in result_empty else "❌ FAIL")
print()

# Test 3: None/missing values in entities
print("Test 3: Malformed Entity Data")
test_malformed = [
    {"suggested_value": "", "column": "BRAND", "similarity_score": 100},
    {"suggested_value": "VALID BRAND", "column": "BRAND", "similarity_score": 90},
]
result_malformed = format_entity_clarification("TEST", test_malformed)
print(f"Input: Entity with empty suggested_value")
print(f"Result: {result_malformed}")
print(f"Result contains 'VALID BRAND': {'VALID BRAND' in result_malformed}")
print(f"Result contains empty value: {result_malformed.count('* **') == 1}")  # Should only have 1 bullet
print(f"✅ PASS: Empty values filtered out" if {'VALID BRAND' in result_malformed} else "❌ FAIL")
print()

# Test 4: Multiple entities with one empty
print("Test 4: Multiple Entities (one empty)")
test_multi_empty = {
    "BRAND": [{"suggested_value": "BRAND 1", "column": "BRAND", "similarity_score": 100}],
    "EMPTY": []
}
result_multi_empty = format_multiple_entities_clarification(test_multi_empty)
print(f"Input: One entity with results, one empty")
print(f"Result: {result_multi_empty[:100]}...")
print(f"✅ PASS: Empty entity filtered" if "EMPTY" not in result_multi_empty else "❌ FAIL")
print()

# Test 5: All entities empty
print("Test 5: All Entities Empty")
test_all_empty = {
    "BRAND1": [],
    "BRAND2": []
}
result_all_empty = format_multiple_entities_clarification(test_all_empty)
print(f"Input: All entities empty")
print(f"Result: {result_all_empty}")
print(f"✅ PASS: Fallback message shown" if "couldn't find any" in result_all_empty else "❌ FAIL")
print()

# Test 6: Entity with missing 'column' field
print("Test 6: Missing Column Field")
test_no_column = [
    {"suggested_value": "BRAND 1", "similarity_score": 100},  # No column field
]
result_no_column = format_entity_clarification("TEST", test_no_column)
print(f"Input: Entity without 'column' field")
print(f"Result: {result_no_column}")
print(f"✅ PASS: Handled gracefully" if {"BRAND 1" in result_no_column} else "❌ FAIL")
print()

# Test 7: Unicode/special characters in value
print("Test 7: Unicode Characters")
test_unicode = [
    {"suggested_value": "JOSÉ CUERVO", "column": "BRAND", "similarity_score": 100},
]
result_unicode = format_entity_clarification("JOSE", test_unicode)
print(f"Input: JOSÉ CUERVO, Search: JOSE")
print(f"Result: {result_unicode}")
print(f"✅ PASS: Unicode preserved" if "José" in result_unicode else "❌ FAIL")
print()

# Test 8: Very long search term
print("Test 8: Long Search Term")
test_long = bold_matching_portion("SHORT", "THIS IS A VERY LONG SEARCH TERM THAT DOES NOT MATCH")
print(f"Input: SHORT, Search: (very long term)")
print(f"Result: {test_long}")
print(f"✅ PASS: No bold (no match)" if test_long == "SHORT" else "❌ FAIL")
print()

# Test 9: Case with special regex characters
print("Test 9: Regex Special Characters")
test_regex = bold_matching_portion("BRAND (SPECIAL)", "BRAND (")
print(f"Input: BRAND (SPECIAL), Search: BRAND (")
print(f"Result: {test_regex}")
print(f"✅ PASS: Handled without error" if "BRAND" in test_regex else "❌ FAIL")
print()

# Test 10: Threshold filtering
print("Test 10: Minimum Threshold Filtering")
test_threshold = [
    {"suggested_value": "HIGH SCORE", "column": "BRAND", "similarity_score": 100},
    {"suggested_value": "LOW SCORE", "column": "BRAND", "similarity_score": 10},
]
result_threshold = get_top_matches(test_threshold, limit=5, min_threshold=50)
print(f"Input: Scores 100 and 10, threshold=50")
print(f"Results count: {len(result_threshold)}")
print(f"Contains HIGH SCORE: {any(e['suggested_value'] == 'HIGH SCORE' for e in result_threshold)}")
print(f"Contains LOW SCORE: {any(e['suggested_value'] == 'LOW SCORE' for e in result_threshold)}")
print(f"✅ PASS: Low score filtered" if len(result_threshold) == 1 else "❌ FAIL")
print()

print("="*80)
print("EDGE CASE VALIDATION COMPLETE")
print("="*80)

EDGE CASE & VALIDATION TESTS

Test 1: Special Characters (apostrophe)
Input: O'DOUL'S AMBER, Search: O'DOUL
Result: **O'DOUL**'S AMBER
✅ PASS: Contains **O'DOUL**

Test 2: Empty Entities List
Input: Empty list
Result: I couldn't find any matches for **"CORONA"** in the data. Please try rephrasing your question or check the spelling.
✅ PASS: Fallback message shown

Test 3: Malformed Entity Data
Input: Entity with empty suggested_value
Result: I couldn't find **"TEST" brand** in the data. Which one did you mean?

* Valid Brand
Result contains 'VALID BRAND': False
Result contains empty value: False
✅ PASS: Empty values filtered out

Test 4: Multiple Entities (one empty)
Input: One entity with results, one empty
Result: I couldn't find **"BRAND" brand** in the data. Which one did you mean?

* **Brand** 1...
✅ PASS: Empty entity filtered

Test 5: All Entities Empty
Input: All entities empty
Result: I couldn't find any matches for the terms **"BRAND1"** and **"BRAND2"** in the data. Please t

In [55]:
# Test Suite for Acceptance Criteria Validation

print("="*80)
print("ACCEPTANCE CRITERIA TEST RESULTS")
print("="*80 + "\n")

# Test 1: Single entity, all same column - Expected header includes column name
test_1_input = {
    "CORONA": [
        {"suggested_value": "CORONA CERO", "column": "BRAND", "similarity_score": 100},
        {"suggested_value": "CORONA EXTRA", "column": "BRAND", "similarity_score": 100},
        {"suggested_value": "CORONA LIGHT", "column": "BRAND", "similarity_score": 100},
    ]
}
result_1 = format_multiple_entities_clarification(test_1_input)
test_1_pass = '**"CORONA" brand**' in result_1 and result_1.count('`brand`') == 0
print(f"✅ Test 1 - Single entity, same column (header): {'PASS' if test_1_pass else 'FAIL'}")
if not test_1_pass:
    print(f"   Expected: Header with 'brand' and no backtick labels")
    print(f"   Got: {result_1[:100]}")

# Test 2: Single entity, mixed columns - Expected column labels on each item
test_2_input = {
    "CORONA": [
        {"suggested_value": "CORONA EXTRA", "column": "BRAND", "similarity_score": 100},
        {"suggested_value": "CORONA LIGHT", "column": "SUBBRAND", "similarity_score": 90},
    ]
}
result_2 = format_multiple_entities_clarification(test_2_input)
test_2_pass = '`brand`' in result_2 and '`subbrand`' in result_2
print(f"✅ Test 2 - Single entity, mixed columns: {'PASS' if test_2_pass else 'FAIL'}")
if not test_2_pass:
    print(f"   Expected: Column labels with backticks")
    print(f"   Got: {result_2}")

# Test 3: Multiple entities - Expected grouping header
test_3_input = {
    "BRAND BB": [
        {"suggested_value": "BRAND", "column": "BRAND", "similarity_score": 100},
    ],
    "REGION N": [
        {"suggested_value": "NORTH REGION", "column": "REGION", "similarity_score": 88},
    ]
}
result_3 = format_multiple_entities_clarification(test_3_input)
test_3_pass = "couldn't find exact matches for some terms" in result_3
print(f"✅ Test 3 - Multiple entities header: {'PASS' if test_3_pass else 'FAIL'}")

# Test 4: Bold matching works
test_4_value = bold_matching_portion("CORONA EXTRA", "CORONA")
test_4_pass = test_4_value == "**CORONA** EXTRA"
print(f"✅ Test 4 - Bold matching portion: {'PASS' if test_4_pass else 'FAIL'}")
if not test_4_pass:
    print(f"   Expected: **CORONA** EXTRA")
    print(f"   Got: {test_4_value}")

# Test 5: Title case formatting
test_5_value = "CORONA EXTRA".title()
test_5_pass = test_5_value == "Corona Extra"
print(f"✅ Test 5 - Title case formatting: {'PASS' if test_5_pass else 'FAIL'}")

# Test 6: Top 5 matches limit
test_6_input = [
    {"suggested_value": f"BRAND_{i}", "column": "BRAND", "similarity_score": 100-i}
    for i in range(10)
]
test_6_result = get_top_matches(test_6_input)
test_6_pass = len(test_6_result) == 5
print(f"✅ Test 6 - Top 5 matches limit: {'PASS' if test_6_pass else 'FAIL'}")

# Test 7: Sorting by similarity score
test_7_input = [
    {"suggested_value": "LOW", "column": "BRAND", "similarity_score": 50},
    {"suggested_value": "HIGH", "column": "BRAND", "similarity_score": 100},
    {"suggested_value": "MED", "column": "BRAND", "similarity_score": 75},
]
test_7_result = get_top_matches(test_7_input)
test_7_pass = test_7_result[0]["suggested_value"] == "HIGH"
print(f"✅ Test 7 - Sorting by similarity score: {'PASS' if test_7_pass else 'FAIL'}")

# Test 8: Alphabetical tie-breaking
test_8_input = [
    {"suggested_value": "ZEBRA", "column": "BRAND", "similarity_score": 100},
    {"suggested_value": "APPLE", "column": "BRAND", "similarity_score": 100},
    {"suggested_value": "BANANA", "column": "BRAND", "similarity_score": 100},
]
test_8_result = get_top_matches(test_8_input)
test_8_pass = test_8_result[0]["suggested_value"] == "APPLE"
print(f"✅ Test 8 - Alphabetical tie-breaking: {'PASS' if test_8_pass else 'FAIL'}")

# Test 9: Case-insensitive bold matching
test_9_value = bold_matching_portion("CORONA EXTRA", "corona")
test_9_pass = "**CORONA**" in test_9_value
print(f"✅ Test 9 - Case-insensitive matching: {'PASS' if test_9_pass else 'FAIL'}")

# Test 10: Partial match in middle
test_10_value = bold_matching_portion("STELLA ARTOIS", "ARTOIS")
test_10_pass = "**ARTOIS**" in test_10_value
print(f"✅ Test 10 - Partial match in middle: {'PASS' if test_10_pass else 'FAIL'}")

# Test 11: Column info detection - same column
test_11_entities = [
    {"column": "BRAND", "suggested_value": "A", "similarity_score": 100},
    {"column": "BRAND", "suggested_value": "B", "similarity_score": 90},
]
test_11_all_same, test_11_col = get_column_info(test_11_entities)
test_11_pass = test_11_all_same and test_11_col == "brand"
print(f"✅ Test 11 - Column info (same): {'PASS' if test_11_pass else 'FAIL'}")

# Test 12: Column info detection - mixed columns
test_12_entities = [
    {"column": "BRAND", "suggested_value": "A", "similarity_score": 100},
    {"column": "SUBBRAND", "suggested_value": "B", "similarity_score": 90},
]
test_12_all_same, test_12_col = get_column_info(test_12_entities)
test_12_pass = not test_12_all_same and test_12_col is None
print(f"✅ Test 12 - Column info (mixed): {'PASS' if test_12_pass else 'FAIL'}")

# Test 13: Bullet list format
test_13_entities = [
    {"suggested_value": "CORONA EXTRA", "column": "BRAND", "similarity_score": 100},
]
test_13_result = format_bullet_list(test_13_entities, "CORONA", True)
test_13_pass = test_13_result.startswith("*") and "**Corona**" in test_13_result
print(f"✅ Test 13 - Bullet list format: {'PASS' if test_13_pass else 'FAIL'}")

# Test 14: No match scenario (no bold)
test_14_value = bold_matching_portion("STELLA ARTOIS", "HEINEKEN")
test_14_pass = test_14_value == "STELLA ARTOIS" and "**" not in test_14_value
print(f"✅ Test 14 - No match (no bold): {'PASS' if test_14_pass else 'FAIL'}")

# Test 15: Special characters in match
test_15_value = bold_matching_portion("O'DOUL'S AMBER", "O'DOUL")
test_15_pass = "**O'DOUL**" in test_15_value
print(f"✅ Test 15 - Special characters: {'PASS' if test_15_pass else 'FAIL'}")

print("\n" + "="*80)
print("SUMMARY")
print("="*80)

all_tests = [test_1_pass, test_2_pass, test_3_pass, test_4_pass, test_5_pass,
             test_6_pass, test_7_pass, test_8_pass, test_9_pass, test_10_pass,
             test_11_pass, test_12_pass, test_13_pass, test_14_pass, test_15_pass]

passed = sum(all_tests)
total = len(all_tests)

print(f"\nTotal Tests: {total}")
print(f"Passed: {passed}")
print(f"Failed: {total - passed}")
print(f"Pass Rate: {passed/total*100:.1f}%")

if passed == total:
    print("\n🎉 All tests passed! Code meets acceptance criteria.")
else:
    print(f"\n⚠️  {total - passed} test(s) failed. Review output above for details.")


ACCEPTANCE CRITERIA TEST RESULTS

✅ Test 1 - Single entity, same column (header): PASS
✅ Test 2 - Single entity, mixed columns: PASS
✅ Test 3 - Multiple entities header: PASS
✅ Test 4 - Bold matching portion: PASS
✅ Test 5 - Title case formatting: PASS
✅ Test 6 - Top 5 matches limit: PASS
✅ Test 7 - Sorting by similarity score: PASS
✅ Test 8 - Alphabetical tie-breaking: PASS
✅ Test 9 - Case-insensitive matching: PASS
✅ Test 10 - Partial match in middle: PASS
✅ Test 11 - Column info (same): PASS
✅ Test 12 - Column info (mixed): PASS
✅ Test 13 - Bullet list format: PASS
✅ Test 14 - No match (no bold): PASS
✅ Test 15 - Special characters: PASS

SUMMARY

Total Tests: 15
Passed: 15
Failed: 0
Pass Rate: 100.0%

🎉 All tests passed! Code meets acceptance criteria.


# Acceptance Criteria Test Report
## User Story 142781: Entity Clarification

**Test Date:** January 8, 2026  
**Test Status:** ✅ PASSED (15/15 tests)

---

## Test Results Summary

| Category | Tests | Passed | Failed | Status |
|----------|-------|--------|--------|--------|
| Functional Requirements | 3 | 3 | 0 | ✅ PASS |
| Text Formatting | 5 | 5 | 0 | ✅ PASS |
| Data Processing | 4 | 4 | 0 | ✅ PASS |
| Edge Cases | 3 | 3 | 0 | ✅ PASS |
| **TOTAL** | **15** | **15** | **0** | ✅ **PASS** |

---

## Detailed Test Results

### ✅ Functional Requirements (3/3)

1. **Single entity, same column** - ✅ PASS
   - Expected: Header includes column name, no backtick labels on items
   - Result: `"CORONA" brand in the data. Which one did you mean?`
   - Items display without column labels (all are brands)

2. **Single entity, mixed columns** - ✅ PASS
   - Expected: Column labels shown with backticks on each item
   - Result: Items correctly show `` `brand` `` and `` `subbrand` `` labels

3. **Multiple entities** - ✅ PASS
   - Expected: Grouped header message
   - Result: `"I couldn't find exact matches for some terms in your question..."`

### ✅ Text Formatting (5/5)

4. **Bold matching portion** - ✅ PASS
   - Input: "CORONA EXTRA" with search term "CORONA"
   - Result: `**CORONA** EXTRA`

5. **Title case formatting** - ✅ PASS
   - Input: "CORONA EXTRA"
   - Result: "Corona Extra"

9. **Case-insensitive matching** - ✅ PASS
   - Input: "CORONA EXTRA" with search term "corona" (lowercase)
   - Result: `**CORONA**` still bolded correctly

10. **Partial match in middle** - ✅ PASS
    - Input: "STELLA ARTOIS" with search term "ARTOIS"
    - Result: `STELLA **ARTOIS**`

13. **Bullet list format** - ✅ PASS
    - Result: Lines start with `*` and include bold text

### ✅ Data Processing (4/4)

6. **Top 5 matches limit** - ✅ PASS
   - Input: 10 entities
   - Result: Exactly 5 returned

7. **Sorting by similarity score** - ✅ PASS
   - Input: Entities with scores 50, 100, 75
   - Result: First item has score 100

8. **Alphabetical tie-breaking** - ✅ PASS
   - Input: "ZEBRA", "APPLE", "BANANA" (all score 100)
   - Result: "APPLE" comes first

11. **Column info detection (same)** - ✅ PASS
    - Input: All entities with column "BRAND"
    - Result: `all_same_column=True, column_name="brand"`

12. **Column info detection (mixed)** - ✅ PASS
    - Input: Mixed "BRAND" and "SUBBRAND"
    - Result: `all_same_column=False, column_name=None`

### ✅ Edge Cases (3/3)

14. **No match scenario** - ✅ PASS
    - Input: "STELLA ARTOIS" with search term "HEINEKEN"
    - Result: No bold applied, returns original text

15. **Special characters** - ✅ PASS
    - Input: "O'DOUL'S AMBER" with search term "O'DOUL"
    - Result: `**O'DOUL**'S AMBER`

---

## Code Coverage

All core functions tested:
- ✅ `bold_matching_portion()`
- ✅ `get_top_matches()`
- ✅ `get_column_info()`
- ✅ `format_bullet_list()`
- ✅ `format_entity_clarification()`
- ✅ `format_multiple_entities_clarification()`

---

## Known Limitations

1. **Unicode/International characters** - Not tested (requires additional test data)
2. **Performance benchmarks** - Not measured (requires timing tests)
3. **User response parsing** - Not implemented yet (separate function needed)

---

## Recommendations

1. ✅ **Ready for integration** - All core acceptance criteria met
2. 📋 **Next steps:**
   - Add unit tests for international characters (accents, Chinese, Arabic)
   - Implement user response parsing function
   - Add performance timing tests (<500ms requirement)
   - Create integration tests with actual backend API

---

## Conclusion

**Status: ✅ APPROVED FOR DELIVERY**

The code successfully meets all testable acceptance criteria for User Story 142781. The entity clarification functionality is working as expected with proper formatting, sorting, and edge case handling.

In [51]:
def format_ambiguity_clarification(data_ambiguity_suggestions):
    """
    Format ambiguity clarification message for terms that exist in multiple categories.
    
    Args:
        data_ambiguity_suggestions: dict - Dictionary with ambiguous terms as keys and list of categories as values
            Example: {'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_VBS_CATEGORY']}
    
    Returns:
        str - Formatted clarification message
    """
    if not data_ambiguity_suggestions:
        return ""
    
    # Filter out any empty values
    filtered_ambiguities = {k: v for k, v in data_ambiguity_suggestions.items() if v and len(v) > 1}
    
    if not filtered_ambiguities:
        return ""
    
    # Single ambiguity case
    if len(filtered_ambiguities) == 1:
        term, categories = next(iter(filtered_ambiguities.items()))
        term_formatted = term.title()
        category_list = format_category_list(categories)
        
        return f'I need some more clarity regarding the term **"{term_formatted}"**. Are you referring to a {category_list}?'
    
    # Multiple ambiguities case
    message = "I need some clarity on some terms in your question. Please clarify them so I can proceed.\n\n"
    
    questions = []
    for term, categories in filtered_ambiguities.items():
        term_formatted = term.title()
        category_list = format_category_list(categories)
        questions.append(f'Is **"{term_formatted}"** a {category_list}?')
    
    return message + "\n\n".join(questions)


# Test with examples
if __name__ == "__main__":
    print("="*80)
    print("AMBIGUITY CLARIFICATION TESTS")
    print("="*80 + "\n")
    
    # Test Case 1: Single ambiguity
    print("Test Case 1: Single Ambiguity")
    print("-" * 60)
    test_1 = {
        'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_VBS_CATEGORY']
    }
    result_1 = format_ambiguity_clarification(test_1)
    print(result_1)
    display(Markdown(result_1))
    print()
    
    # Test Case 2: Multiple ambiguities
    print("\n" + "="*80)
    print("Test Case 2: Multiple Ambiguities")
    print("-" * 60)
    test_2 = {
        'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_VBS_CATEGORY'],
        'SERVICES': ['IT_SERVICES', 'CONSULTING_SERVICES', 'MANAGED_SERVICES']
    }
    result_2 = format_ambiguity_clarification(test_2)
    print(result_2)
    display(Markdown(result_2))
    print()
    
    # Test Case 3: Three categories
    print("\n" + "="*80)
    print("Test Case 3: Three or More Categories")
    print("-" * 60)
    test_3 = {
        'BRAND': ['BEER_BRAND', 'SPIRITS_BRAND', 'WINE_BRAND', 'NON_ALCOHOLIC_BRAND']
    }
    result_3 = format_ambiguity_clarification(test_3)
    print(result_3)
    display(Markdown(result_3))
    print()
    
    # Test Case 4: Empty input
    print("\n" + "="*80)
    print("Test Case 4: Empty Input (Edge Case)")
    print("-" * 60)
    test_4 = {}
    result_4 = format_ambiguity_clarification(test_4)
    print(f"Result: '{result_4}' (empty string expected)")
    print(f"✅ PASS" if result_4 == "" else "❌ FAIL")
    print()
    
    # Test Case 5: Full response object from API
    print("\n" + "="*80)
    print("Test Case 5: Full API Response Object")
    print("-" * 60)
    full_response = {
        'extracted_entities': {'GPO_CATEGORY': ['TECHNOLOGY']},
        'validated_entities': {'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_VBS_CATEGORY']},
        'suggested_entities': {},
        'data_ambiguity_suggestions': {'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_VBS_CATEGORY']},
        'unknown_entities': [],
        'hil_trigger': True,
        'ner_augmented_question': 'What is the current spend on technology for ABInBev in year 2025?',
        'human_question': 'What is the current spend on technology for ABInBev in year 2025?',
        'suggested_questions': [
            'what is the current spend on GPO_CATEGORY technology for abinbev in year 2025?',
            'what is the current spend on VENDOR_VBS_CATEGORY technology for abinbev in year 2025?'
        ],
        'response': "There is ambiguity in data for the following entities: 'TECHNOLOGY' is present in both GPO_CATEGORY and VENDOR_VBS_CATEGORY."
    }
    result_5 = format_ambiguity_clarification(full_response.get('data_ambiguity_suggestions', {}))
    print(result_5)
    display(Markdown(result_5))
    print()


AMBIGUITY CLARIFICATION TESTS

Test Case 1: Single Ambiguity
------------------------------------------------------------
I need some more clarity regarding the term **"Technology"**. Are you referring to a `gpo category` or `vendor vbs category`?


I need some more clarity regarding the term **"Technology"**. Are you referring to a `gpo category` or `vendor vbs category`?



Test Case 2: Multiple Ambiguities
------------------------------------------------------------
I need some clarity on some terms in your question. Please clarify them so I can proceed.

Is **"Technology"** a `gpo category` or `vendor vbs category`?

Is **"Services"** a `it services`, `consulting services`, or `managed services`?


I need some clarity on some terms in your question. Please clarify them so I can proceed.

Is **"Technology"** a `gpo category` or `vendor vbs category`?

Is **"Services"** a `it services`, `consulting services`, or `managed services`?



Test Case 3: Three or More Categories
------------------------------------------------------------
I need some more clarity regarding the term **"Brand"**. Are you referring to a `beer brand`, `spirits brand`, `wine brand`, or `non alcoholic brand`?


I need some more clarity regarding the term **"Brand"**. Are you referring to a `beer brand`, `spirits brand`, `wine brand`, or `non alcoholic brand`?



Test Case 4: Empty Input (Edge Case)
------------------------------------------------------------
Result: '' (empty string expected)
✅ PASS


Test Case 5: Full API Response Object
------------------------------------------------------------
I need some more clarity regarding the term **"Technology"**. Are you referring to a `gpo category` or `vendor vbs category`?


I need some more clarity regarding the term **"Technology"**. Are you referring to a `gpo category` or `vendor vbs category`?

In [52]:
# Validation Tests for Ambiguity Clarification

print("="*80)
print("AMBIGUITY CLARIFICATION - ACCEPTANCE CRITERIA VALIDATION")
print("="*80 + "\n")

# Test 1: Single ambiguity format
test_1_input = {'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_VBS_CATEGORY']}
result_1 = format_ambiguity_clarification(test_1_input)
test_1_pass = (
    '**"Technology"**' in result_1 and
    'I need some more clarity regarding the term' in result_1 and
    '`gpo category`' in result_1 and
    '`vendor vbs category`' in result_1 and
    'Are you referring to a' in result_1
)
print(f"✅ Test 1 - Single ambiguity format: {'PASS' if test_1_pass else 'FAIL'}")
if not test_1_pass:
    print(f"   Got: {result_1}")

# Test 2: Multiple ambiguities format
test_2_input = {
    'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_VBS_CATEGORY'],
    'BRAND': ['BEER_BRAND', 'SPIRITS_BRAND']
}
result_2 = format_ambiguity_clarification(test_2_input)
test_2_pass = (
    'I need some clarity on some terms' in result_2 and
    'Is **"Technology"**' in result_2 and
    'Is **"Brand"**' in result_2 and
    result_2.count('Is **"') == 2
)
print(f"✅ Test 2 - Multiple ambiguities format: {'PASS' if test_2_pass else 'FAIL'}")
if not test_2_pass:
    print(f"   Got: {result_2}")

# Test 3: Two categories grammar
test_3_input = {'TERM': ['CATEGORY_1', 'CATEGORY_2']}
result_3 = format_ambiguity_clarification(test_3_input)
test_3_pass = '`category 1` or `category 2`' in result_3
print(f"✅ Test 3 - Two categories grammar (or): {'PASS' if test_3_pass else 'FAIL'}")
if not test_3_pass:
    print(f"   Expected: 'or' between two categories")
    print(f"   Got: {result_3}")

# Test 4: Three+ categories grammar
test_4_input = {'TERM': ['CAT_1', 'CAT_2', 'CAT_3']}
result_4 = format_ambiguity_clarification(test_4_input)
test_4_pass = '`cat 1`, `cat 2`, or `cat 3`' in result_4
print(f"✅ Test 4 - Three+ categories grammar (commas + or): {'PASS' if test_4_pass else 'FAIL'}")
if not test_4_pass:
    print(f"   Expected: commas with 'or' before last")
    print(f"   Got: {result_4}")

# Test 5: Term formatting (title case)
test_5_input = {'TECHNOLOGY': ['CAT_1', 'CAT_2']}
result_5 = format_ambiguity_clarification(test_5_input)
test_5_pass = '**"Technology"**' in result_5
print(f"✅ Test 5 - Term title case: {'PASS' if test_5_pass else 'FAIL'}")
if not test_5_pass:
    print(f"   Expected: **\"Technology\"** (title case)")
    print(f"   Got: {result_5}")

# Test 6: Category formatting (lowercase, underscores to spaces)
test_6_input = {'TERM': ['GPO_CATEGORY', 'VENDOR_VBS_CATEGORY']}
result_6 = format_ambiguity_clarification(test_6_input)
test_6_pass = '`gpo category`' in result_6 and '`vendor vbs category`' in result_6
print(f"✅ Test 6 - Category formatting: {'PASS' if test_6_pass else 'FAIL'}")
if not test_6_pass:
    print(f"   Expected: lowercase with spaces")
    print(f"   Got: {result_6}")

# Test 7: Empty input handling
test_7_input = {}
result_7 = format_ambiguity_clarification(test_7_input)
test_7_pass = result_7 == ""
print(f"✅ Test 7 - Empty input returns empty string: {'PASS' if test_7_pass else 'FAIL'}")

# Test 8: Single category (should filter out - not ambiguous)
test_8_input = {'TERM': ['ONLY_ONE_CATEGORY']}
result_8 = format_ambiguity_clarification(test_8_input)
test_8_pass = result_8 == ""
print(f"✅ Test 8 - Single category filtered: {'PASS' if test_8_pass else 'FAIL'}")
if not test_8_pass:
    print(f"   Expected: Empty string (not ambiguous)")
    print(f"   Got: {result_8}")

# Test 9: Markdown rendering
test_9_input = {'BRAND': ['BEER', 'WINE']}
result_9 = format_ambiguity_clarification(test_9_input)
test_9_pass = '**"Brand"**' in result_9 and '`beer`' in result_9
print(f"✅ Test 9 - Markdown formatting: {'PASS' if test_9_pass else 'FAIL'}")

# Test 10: Real API response structure
test_10_full_response = {
    'data_ambiguity_suggestions': {'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_VBS_CATEGORY']}
}
result_10 = format_ambiguity_clarification(test_10_full_response.get('data_ambiguity_suggestions', {}))
test_10_pass = '**"Technology"**' in result_10 and 'gpo category' in result_10
print(f"✅ Test 10 - API response object handling: {'PASS' if test_10_pass else 'FAIL'}")

print("\n" + "="*80)
print("SUMMARY")
print("="*80)

all_tests = [test_1_pass, test_2_pass, test_3_pass, test_4_pass, test_5_pass,
             test_6_pass, test_7_pass, test_8_pass, test_9_pass, test_10_pass]

passed = sum(all_tests)
total = len(all_tests)

print(f"\nTotal Tests: {total}")
print(f"Passed: {passed}")
print(f"Failed: {total - passed}")
print(f"Pass Rate: {passed/total*100:.1f}%")

if passed == total:
    print("\n🎉 All ambiguity clarification tests passed!")
else:
    print(f"\n⚠️  {total - passed} test(s) failed. Review output above for details.")

AMBIGUITY CLARIFICATION - ACCEPTANCE CRITERIA VALIDATION

✅ Test 1 - Single ambiguity format: PASS
✅ Test 2 - Multiple ambiguities format: PASS
✅ Test 3 - Two categories grammar (or): PASS
✅ Test 4 - Three+ categories grammar (commas + or): PASS
✅ Test 5 - Term title case: PASS
✅ Test 6 - Category formatting: PASS
✅ Test 7 - Empty input returns empty string: PASS
✅ Test 8 - Single category filtered: PASS
✅ Test 9 - Markdown formatting: PASS
✅ Test 10 - API response object handling: PASS

SUMMARY

Total Tests: 10
Passed: 10
Failed: 0
Pass Rate: 100.0%

🎉 All ambiguity clarification tests passed!


## Ambiguity Clarification - Example Outputs

### Single Ambiguity Example

In [53]:
# Example 1: Single Ambiguous Term
from IPython.display import Markdown, display

example_1 = {'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_VBS_CATEGORY']}
output_1 = format_ambiguity_clarification(example_1)
print("Input:")
print(example_1)
print("\nOutput:")
display(Markdown(output_1))

Input:
{'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_VBS_CATEGORY']}

Output:


I need some more clarity regarding the term **"Technology"**. Are you referring to a `gpo category` or `vendor vbs category`?

In [54]:
# Example 2: Multiple Ambiguous Terms
example_2 = {
    'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_VBS_CATEGORY'],
    'SERVICES': ['GPO_SERVICES', 'VENDOR_SERVICES'],
    'BRAND': ['BEER_BRAND', 'SPIRITS_BRAND', 'WINE_BRAND']
}
output_2 = format_ambiguity_clarification(example_2)
print("Input:")
print(example_2)
print("\nOutput:")
display(Markdown(output_2))

Input:
{'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_VBS_CATEGORY'], 'SERVICES': ['GPO_SERVICES', 'VENDOR_SERVICES'], 'BRAND': ['BEER_BRAND', 'SPIRITS_BRAND', 'WINE_BRAND']}

Output:


I need some clarity on some terms in your question. Please clarify them so I can proceed.

Is **"Technology"** a `gpo category` or `vendor vbs category`?

Is **"Services"** a `gpo services` or `vendor services`?

Is **"Brand"** a `beer brand`, `spirits brand`, or `wine brand`?

In [64]:
def format_unified_clarification(suggested_entities=None, data_ambiguity_suggestions=None):
    """
    Unified function to handle both entity suggestions and ambiguities.
    
    Args:
        suggested_entities: dict - Dictionary with user inputs as keys and suggested entities as values
            Example: {"CORONA": [{"suggested_value": "CORONA EXTRA", "column": "BRAND", "similarity_score": 100}]}
        data_ambiguity_suggestions: dict - Dictionary with ambiguous terms as keys and list of categories as values
            Example: {'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_CATEGORY']}
    
    Returns:
        str - Formatted unified clarification message
    """
    # Convert ambiguities to suggestion format
    ambiguity_as_suggestions = {}
    if data_ambiguity_suggestions:
        for term, categories in data_ambiguity_suggestions.items():
            # Filter out single-category items (not ambiguous)
            if len(categories) > 1:
                # Convert categories to suggestion format
                ambiguity_as_suggestions[term] = [
                    {
                        "suggested_value": term,
                        "column": category,
                        "similarity_score": 100
                    }
                    for category in categories
                ]
    
    # Combine both dictionaries
    combined_dict = {}
    if suggested_entities:
        combined_dict.update(suggested_entities)
    if ambiguity_as_suggestions:
        combined_dict.update(ambiguity_as_suggestions)
    
    # Use existing function to format
    if not combined_dict:
        return ""
    
    # Filter out empty entity lists
    filtered_dict = {k: v for k, v in combined_dict.items() if v}
    
    if not filtered_dict:
        return ""
    
    if len(filtered_dict) == 1:
        # Single entity/ambiguity case
        user_input, entities = next(iter(filtered_dict.items()))
        
        # Check if all entities have same suggested_value (ambiguity case)
        unique_values = set(e.get("suggested_value", "") for e in entities)
        is_ambiguity = len(unique_values) == 1 and user_input.upper() == list(unique_values)[0].upper()
        
        top_matches = get_top_matches(entities)
        if not top_matches:
            return f'I couldn\'t find any matches for **"{user_input}"** in the data. Please try rephrasing your question or check the spelling.'
        
        # For single item, don't show "some terms", just show the direct question
        if is_ambiguity:
            # Ambiguity: show bullet list with term repeated (always show columns for ambiguity)
            header = f'I need some more clarity regarding the term **"{user_input.title()}"**. Please clarify which one you mean.\n\n'
            all_same_column = False  # Always show columns for ambiguities
        else:
            # Suggestion: check if all same column
            all_same_column, column_name = get_column_info(top_matches)
            if all_same_column and column_name:
                column_formatted = column_name.replace('_', ' ')
                header = f'I couldn\'t find **"{user_input}" {column_formatted}** in the data. Which one did you mean?\n\n'
            else:
                header = f'I couldn\'t find **"{user_input}"** in the data. Which one did you mean?\n\n'
        
        # Build bullet list (use all_same_column to control label display)
        bullet_list = format_bullet_list(top_matches, user_input, all_same_column)
        
        if not bullet_list:
            return f'I couldn\'t find any valid matches for **"{user_input}"** in the data. Please try rephrasing your question.'
        
        return header + bullet_list
    
    # Multiple entities case
    message = "I need clarity on some terms in your question. Please clarify them so I can proceed.\n\n"
    
    sections = []
    for user_input, entities in filtered_dict.items():
        top_matches = get_top_matches(entities)
        
        if not top_matches:
            continue
        
        # Check if this is an ambiguity (all same suggested_value)
        unique_values = set(e.get("suggested_value", "") for e in top_matches)
        is_ambiguity = len(unique_values) == 1 and user_input.upper() == list(unique_values)[0].upper()
        
        all_same_column, column_name = get_column_info(top_matches)
        
        # Build section header
        if not is_ambiguity and column_name and all_same_column:
            column_formatted = column_name.replace('_', ' ')
            section_header = f'**"{user_input}" {column_formatted}**:'
        else:
            section_header = f'**"{user_input.title()}"**:'
        # Build bullet list (use all_same_column, but force False for ambiguities)
        use_all_same = all_same_column if not is_ambiguity else False
        bullet_list = format_bullet_list(top_matches, user_input, use_all_same)
        
        if bullet_list:
            sections.append(f"{section_header}\n{bullet_list}")
    
    if not sections:
        return "I couldn't find any valid matches for the terms in your question. Please try rephrasing or check the spelling."
    
    return message + "\n\n".join(sections)


# Test all 5 cases
if __name__ == "__main__":
    print("="*80)
    print("UNIFIED CLARIFICATION - ALL TEST CASES")
    print("="*80 + "\n")
    
    # Case 1: Only one suggestion
    print("Case 1: Only One Suggestion")
    print("-" * 60)
    case_1_suggestions = {
        "CORONA": [
            {"suggested_value": "CORONA CERO", "column": "BRAND", "similarity_score": 100},
            {"suggested_value": "CORONA EXTRA", "column": "BRAND", "similarity_score": 100},
            {"suggested_value": "CORONA GOLDEN LIGHT", "column": "SUBBRAND", "similarity_score": 100},
            {"suggested_value": "CORONA LIGERA", "column": "BRAND", "similarity_score": 100},
            {"suggested_value": "CORONA LIGHT", "column": "BRAND", "similarity_score": 100},
        ]
    }
    result_1 = format_unified_clarification(suggested_entities=case_1_suggestions)
    print(result_1)
    display(Markdown(result_1))
    print("\n")
    
    # Case 2: Only one ambiguity
    print("="*80)
    print("Case 2: Only One Ambiguity")
    print("-" * 60)
    case_2_ambiguity = {'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_CATEGORY', 'VENDOR_VBS_CATEGORY']}
    result_2 = format_unified_clarification(data_ambiguity_suggestions=case_2_ambiguity)
    print(result_2)
    display(Markdown(result_2))
    print("\n")
    
    # Case 3: More than one suggestions
    print("="*80)
    print("Case 3: More Than One Suggestions")
    print("-" * 60)
    case_3_suggestions = {
        "BRAND BB": [
            {"suggested_value": "BRAND", "column": "BRAND", "similarity_score": 100},
            {"suggested_value": "ANDES", "column": "SUBBRAND", "similarity_score": 75},
            {"suggested_value": "ANOTHER BRAND", "column": "BRAND", "similarity_score": 75},
        ],
        "REGION N": [
            {"suggested_value": "CENTRAL REGION", "column": "REGION", "similarity_score": 88},
            {"suggested_value": "NORTH EAST REGION", "column": "REGION", "similarity_score": 88},
            {"suggested_value": "NORTH REGION", "column": "REGION", "similarity_score": 100},
            {"suggested_value": "NORTH WEST REGION", "column": "REGION", "similarity_score": 88},
            {"suggested_value": "REGION PARISIENNE", "column": "REGION", "similarity_score": 88},
        ]
    }
    result_3 = format_unified_clarification(suggested_entities=case_3_suggestions)
    print(result_3)
    display(Markdown(result_3))
    print("\n")
    
    # Case 4: More than one ambiguities
    print("="*80)
    print("Case 4: More Than One Ambiguities")
    print("-" * 60)
    case_4_ambiguity = {
        'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_CATEGORY'],
        'BRAND': ['BEER_BRAND', 'SPIRITS_BRAND', 'WINE_BRAND']
    }
    result_4 = format_unified_clarification(data_ambiguity_suggestions=case_4_ambiguity)
    print(result_4)
    display(Markdown(result_4))
    print("\n")
    
    # Case 5: Mixed ambiguities and suggestions
    print("="*80)
    print("Case 5: Mixed Ambiguities and Suggestions")
    print("-" * 60)
    case_5_suggestions = {
        "REGION N": [
            {"suggested_value": "CENTRAL REGION", "column": "REGION", "similarity_score": 88},
            {"suggested_value": "NORTH REGION", "column": "REGION", "similarity_score": 100},
            {"suggested_value": "NORTH WEST REGION", "column": "REGION", "similarity_score": 88},
        ]
    }
    case_5_ambiguity = {
        'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_CATEGORY'],
        'BRAND': ['BEER_BRAND', 'SPIRITS_BRAND', 'WINE_BRAND']
    }
    result_5 = format_unified_clarification(
        suggested_entities=case_5_suggestions, 
        data_ambiguity_suggestions=case_5_ambiguity
    )
    print(result_5)
    display(Markdown(result_5))
    print("\n")

UNIFIED CLARIFICATION - ALL TEST CASES

Case 1: Only One Suggestion
------------------------------------------------------------
I couldn't find **"CORONA"** in the data. Which one did you mean?

* **Corona** Cero `brand`
* **Corona** Extra `brand`
* **Corona** Golden Light `subbrand`
* **Corona** Ligera `brand`
* **Corona** Light `brand`


I couldn't find **"CORONA"** in the data. Which one did you mean?

* **Corona** Cero `brand`
* **Corona** Extra `brand`
* **Corona** Golden Light `subbrand`
* **Corona** Ligera `brand`
* **Corona** Light `brand`



Case 2: Only One Ambiguity
------------------------------------------------------------
I need some more clarity regarding the term **"Technology"**. Please clarify which one you mean.

* **Technology** `gpo category`
* **Technology** `vendor category`
* **Technology** `vendor vbs category`


I need some more clarity regarding the term **"Technology"**. Please clarify which one you mean.

* **Technology** `gpo category`
* **Technology** `vendor category`
* **Technology** `vendor vbs category`



Case 3: More Than One Suggestions
------------------------------------------------------------
I need clarity on some terms in your question. Please clarify them so I can proceed.

**"Brand Bb"**:
* Brand `brand`
* Andes `subbrand`
* Another Brand `brand`

**"REGION N" region**:
* North Region
* Central Region
* North East Region
* North West Region
* Region Parisienne


I need clarity on some terms in your question. Please clarify them so I can proceed.

**"Brand Bb"**:
* Brand `brand`
* Andes `subbrand`
* Another Brand `brand`

**"REGION N" region**:
* North Region
* Central Region
* North East Region
* North West Region
* Region Parisienne



Case 4: More Than One Ambiguities
------------------------------------------------------------
I need clarity on some terms in your question. Please clarify them so I can proceed.

**"Technology"**:
* **Technology** `gpo category`
* **Technology** `vendor category`

**"Brand"**:
* **Brand** `beer brand`
* **Brand** `spirits brand`
* **Brand** `wine brand`


I need clarity on some terms in your question. Please clarify them so I can proceed.

**"Technology"**:
* **Technology** `gpo category`
* **Technology** `vendor category`

**"Brand"**:
* **Brand** `beer brand`
* **Brand** `spirits brand`
* **Brand** `wine brand`



Case 5: Mixed Ambiguities and Suggestions
------------------------------------------------------------
I need clarity on some terms in your question. Please clarify them so I can proceed.

**"REGION N" region**:
* North Region
* Central Region
* North West Region

**"Technology"**:
* **Technology** `gpo category`
* **Technology** `vendor category`

**"Brand"**:
* **Brand** `beer brand`
* **Brand** `spirits brand`
* **Brand** `wine brand`


I need clarity on some terms in your question. Please clarify them so I can proceed.

**"REGION N" region**:
* North Region
* Central Region
* North West Region

**"Technology"**:
* **Technology** `gpo category`
* **Technology** `vendor category`

**"Brand"**:
* **Brand** `beer brand`
* **Brand** `spirits brand`
* **Brand** `wine brand`

In [66]:
# Comprehensive Test Suite for Unified Clarification

print("="*80)
print("UNIFIED CLARIFICATION - ACCEPTANCE TESTS")
print("="*80 + "\n")

# Test 1: Single suggestion - no column labels when not mixed
test_1 = format_unified_clarification(
    suggested_entities={"CORONA": [
        {"suggested_value": "CORONA CERO", "column": "BRAND", "similarity_score": 100},
        {"suggested_value": "CORONA EXTRA", "column": "BRAND", "similarity_score": 100}
    ]}
)
test_1_pass = 'I couldn\'t find **"CORONA"' in test_1 and '* **Corona**' in test_1
print(f"✅ Test 1 - Single suggestion format: {'PASS' if test_1_pass else 'FAIL'}")

# Test 2: Single ambiguity - shows term repeated with categories
test_2 = format_unified_clarification(
    data_ambiguity_suggestions={'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_CATEGORY']}
)
test_2_pass = (
    'I need some more clarity regarding the term **"Technology"**' in test_2 and
    '* **Technology** `gpo category`' in test_2 and
    '* **Technology** `vendor category`' in test_2
)
print(f"✅ Test 2 - Single ambiguity format: {'PASS' if test_2_pass else 'FAIL'}")

# Test 3: Multiple suggestions - groups with headers
test_3 = format_unified_clarification(
    suggested_entities={
        "BRAND": [{"suggested_value": "BRAND 1", "column": "BRAND", "similarity_score": 100}],
        "REGION": [{"suggested_value": "REGION 1", "column": "REGION", "similarity_score": 100}]
    }
)
test_3_pass = (
    'I need clarity on some terms' in test_3 and
    '**"BRAND"' in test_3 and  # Check for uppercase in header
    '**"REGION"' in test_3
)
print(f"✅ Test 3 - Multiple suggestions format: {'PASS' if test_3_pass else 'FAIL'}")

# Test 4: Multiple ambiguities - shows all as bullet lists
test_4 = format_unified_clarification(
    data_ambiguity_suggestions={
        'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_CATEGORY'],
        'BRAND': ['BEER_BRAND', 'WINE_BRAND']
    }
)
test_4_pass = (
    '**"Technology"**:' in test_4 and
    '* **Technology** `gpo category`' in test_4 and
    '**"Brand"**:' in test_4 and
    '* **Brand** `beer brand`' in test_4
)
print(f"✅ Test 4 - Multiple ambiguities format: {'PASS' if test_4_pass else 'FAIL'}")

# Test 5: Mixed suggestions and ambiguities
test_5 = format_unified_clarification(
    suggested_entities={"REGION": [
        {"suggested_value": "NORTH REGION", "column": "REGION", "similarity_score": 100}
    ]},
    data_ambiguity_suggestions={'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_CATEGORY']}
)
test_5_pass = (
    'I need clarity on some terms' in test_5 and
    '**"REGION' in test_5 and
    '**"Technology"**:' in test_5 and
    '* **Technology**' in test_5
)
print(f"✅ Test 5 - Mixed suggestions and ambiguities: {'PASS' if test_5_pass else 'FAIL'}")

# Test 6: Empty inputs
test_6 = format_unified_clarification(suggested_entities={}, data_ambiguity_suggestions={})
test_6_pass = test_6 == ""
print(f"✅ Test 6 - Empty inputs return empty string: {'PASS' if test_6_pass else 'FAIL'}")

# Test 7: Single category in ambiguity (should be filtered)
test_7 = format_unified_clarification(
    data_ambiguity_suggestions={'TERM': ['ONLY_ONE_CATEGORY']}
)
test_7_pass = test_7 == ""
print(f"✅ Test 7 - Single category filtered out: {'PASS' if test_7_pass else 'FAIL'}")

# Test 8: Column name formatting (underscores to spaces)
test_8 = format_unified_clarification(
    data_ambiguity_suggestions={'TECH': ['GPO_CATEGORY', 'VENDOR_VBS_CATEGORY']}
)
test_8_pass = '`gpo category`' in test_8 and '`vendor vbs category`' in test_8
print(f"✅ Test 8 - Column name formatting: {'PASS' if test_8_pass else 'FAIL'}")

# Test 9: Title case for terms
test_9 = format_unified_clarification(
    data_ambiguity_suggestions={'TECHNOLOGY': ['CAT1', 'CAT2']}
)
test_9_pass = '**"Technology"**' in test_9
print(f"✅ Test 9 - Term title case: {'PASS' if test_9_pass else 'FAIL'}")

# Test 10: Bold matching in suggestions
test_10 = format_unified_clarification(
    suggested_entities={"CORONA": [
        {"suggested_value": "CORONA EXTRA", "column": "BRAND", "similarity_score": 100}
    ]}
)
test_10_pass = '**Corona**' in test_10
print(f"✅ Test 10 - Bold matching in suggestions: {'PASS' if test_10_pass else 'FAIL'}")

# Test 11: Bold matching in ambiguities
test_11 = format_unified_clarification(
    data_ambiguity_suggestions={'TECHNOLOGY': ['CAT1', 'CAT2']}
)
test_11_pass = '**Technology**' in test_11
print(f"✅ Test 11 - Bold term in ambiguities: {'PASS' if test_11_pass else 'FAIL'}")

# Test 12: All same column in single suggestion - should NOT show column labels
test_12 = format_unified_clarification(
    suggested_entities={"BRAND": [
        {"suggested_value": "BRAND 1", "column": "BRAND", "similarity_score": 100},
        {"suggested_value": "BRAND 2", "column": "BRAND", "similarity_score": 90}
    ]}
)
test_12_pass = '* **Brand** 1' in test_12 and test_12.count('`brand`') == 0  # Should NOT show column labels
print(f"✅ Test 12 - No column labels when all same: {'PASS' if test_12_pass else 'FAIL'}")

# Test 13: Top 5 limit still applies
test_13 = format_unified_clarification(
    suggested_entities={"TERM": [
        {"suggested_value": f"BRAND_{i}", "column": "BRAND", "similarity_score": 100-i}
        for i in range(10)
    ]}
)
# Count lines that start with * (bullet points)
test_13_pass = test_13.count('\n* ') == 5  # Should have exactly 5 bullet lines
print(f"✅ Test 13 - Top 5 limit applied: {'PASS' if test_13_pass else 'FAIL'}")

# Test 14: Only suggestions provided (no ambiguities)
test_14 = format_unified_clarification(
    suggested_entities={"BRAND": [
        {"suggested_value": "BRAND 1", "column": "BRAND", "similarity_score": 100}
    ]},
    data_ambiguity_suggestions=None
)
test_14_pass = test_14 != ""
print(f"✅ Test 14 - Only suggestions (None ambiguities): {'PASS' if test_14_pass else 'FAIL'}")

# Test 15: Only ambiguities provided (no suggestions)
test_15 = format_unified_clarification(
    suggested_entities=None,
    data_ambiguity_suggestions={'TECH': ['CAT1', 'CAT2']}
)
test_15_pass = test_15 != ""
print(f"✅ Test 15 - Only ambiguities (None suggestions): {'PASS' if test_15_pass else 'FAIL'}")

print("\n" + "="*80)
print("SUMMARY")
print("="*80)

all_unified_tests = [
    test_1_pass, test_2_pass, test_3_pass, test_4_pass, test_5_pass,
    test_6_pass, test_7_pass, test_8_pass, test_9_pass, test_10_pass,
    test_11_pass, test_12_pass, test_13_pass, test_14_pass, test_15_pass
]

passed = sum(all_unified_tests)
total = len(all_unified_tests)

print(f"\nTotal Tests: {total}")
print(f"Passed: {passed}")
print(f"Failed: {total - passed}")
print(f"Pass Rate: {passed/total*100:.1f}%")

if passed == total:
    print("\n🎉 All unified clarification tests passed!")
else:
    print(f"\n⚠️  {total - passed} test(s) failed. Review output above for details.")

UNIFIED CLARIFICATION - ACCEPTANCE TESTS

✅ Test 1 - Single suggestion format: PASS
✅ Test 2 - Single ambiguity format: PASS
✅ Test 3 - Multiple suggestions format: PASS
✅ Test 4 - Multiple ambiguities format: PASS
✅ Test 5 - Mixed suggestions and ambiguities: PASS
✅ Test 6 - Empty inputs return empty string: PASS
✅ Test 7 - Single category filtered out: PASS
✅ Test 8 - Column name formatting: PASS
✅ Test 9 - Term title case: PASS
✅ Test 10 - Bold matching in suggestions: PASS
✅ Test 11 - Bold term in ambiguities: PASS
✅ Test 12 - No column labels when all same: PASS
✅ Test 13 - Top 5 limit applied: PASS
✅ Test 14 - Only suggestions (None ambiguities): PASS
✅ Test 15 - Only ambiguities (None suggestions): PASS

SUMMARY

Total Tests: 15
Passed: 15
Failed: 0
Pass Rate: 100.0%

🎉 All unified clarification tests passed!


In [65]:
# Debug Test 12
print("="*80)
print("DEBUG TEST 12")
print("="*80 + "\n")

test_12_debug = format_unified_clarification(
    suggested_entities={"BRAND": [
        {"suggested_value": "BRAND 1", "column": "BRAND", "similarity_score": 100},
        {"suggested_value": "BRAND 2", "column": "BRAND", "similarity_score": 90}
    ]}
)
print("Test 12 Full Output:")
print(test_12_debug)
print("\n" + "="*60)
print(f"Contains '* **Brand** 1': {'* **Brand** 1' in test_12_debug}")
print(f"Count of '`brand`': {test_12_debug.count('`brand`')}")
print(f"Expected: 0 backtick labels")
print(f"Test should PASS if: Contains '* **Brand** 1' AND count is 0")

DEBUG TEST 12

Test 12 Full Output:
I couldn't find **"BRAND" brand** in the data. Which one did you mean?

* **Brand** 1
* **Brand** 2

Contains '* **Brand** 1': True
Count of '`brand`': 0
Expected: 0 backtick labels
Test should PASS if: Contains '* **Brand** 1' AND count is 0


## 🎯 Unified Clarification Function - Complete

### Overview
The `format_unified_clarification()` function combines both entity suggestions (similarity matching) and ambiguity clarification into a single, cohesive user experience.

### Key Features
- **Handles 5 scenarios**: Single suggestion, single ambiguity, multiple suggestions, multiple ambiguities, and mixed cases
- **Consistent formatting**: Both suggestions and ambiguities use bullet list format with column labels
- **Smart detection**: Automatically identifies ambiguities (same term, multiple categories) vs suggestions (different terms)
- **Title case formatting**: User-friendly term display
- **Column name normalization**: Converts `GPO_CATEGORY` → `gpo category`

### Usage

```python
# Case 1: Only suggestions
format_unified_clarification(
    suggested_entities={"CORONA": [...]},
    data_ambiguity_suggestions=None
)

# Case 2: Only ambiguities  
format_unified_clarification(
    suggested_entities=None,
    data_ambiguity_suggestions={'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_CATEGORY']}
)

# Case 3: Mixed (both)
format_unified_clarification(
    suggested_entities={"REGION": [...]},
    data_ambiguity_suggestions={'TECHNOLOGY': ['GPO_CATEGORY', 'VENDOR_CATEGORY']}
)
```

### Test Results
- ✅ **15/15 tests passing (100%)**
- ✅ All 5 scenarios validated
- ✅ Edge cases handled (empty inputs, single categories filtered)
- ✅ Format consistency across all cases